In [ ]:
# Debug RAG

In [ ]:
%%capture
!pip install chainlit

In [5]:
import chainlit as cl 
import chromadb 
import requests 
import os
from chromadb.config import Settings
from datetime import datetime
from chromadb.utils import embedding_functions

api_host = '8080'
api_port = '10.182.1.48'
RAG_DB_PATH='/nvmefs1/test_user/cache/rag_db3/'
EMB_PATH='/nvmefs1/test_user/cache/vector_model/e5-base-v2'
print(api_host,api_port,RAG_DB_PATH,EMB_PATH)
#parser = argparse.ArgumentParser(description="Process a file.") 
#path = '/mnt/efs/shared_fs/determined/rag_db/' 
#db = chromadb.PersistentClient(path=path) 
#collection = db.get_collection("HPE_press_releases") 
#EXTERNAL_IP="10.239.100.80"
#parser.add_argument("external_api_ip", default=None,help="External IP for API") 
#parser.add_argument("path_to_db", default='/mnt/efs/shared_fs/determined/rag_db',help="Path to vector db in container")
#parser.add_argument("host",default="0.0.0.0")
#parser.add_argument("port", default=8085)

#titan_url = "http://{}:8006/generate_stream".format(EXTERNAL_IP)
#args = parser.parse_args()


#path = '/mnt/efs/shared_fs/determined/rag_db/'
path = RAG_DB_PATH
settings = chromadb.get_settings()
settings.allow_reset = True
db = chromadb.PersistentClient(path=path,settings=settings)
emb_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMB_PATH, device="cpu"
)
collection = db.get_collection("HPE_press_releases",embedding_function=emb_fn)
#parser.add_argument("filepath", help="Path to the XML file")
titan_url = "http://{}:{}/generate_stream".format(api_host,api_port)
print("Titan URL: ",titan_url)



8080 10.182.1.48 /nvmefs1/test_user/cache/rag_db3/ /nvmefs1/test_user/cache/vector_model/e5-base-v2
Titan URL:  http://8080:10.182.1.48/generate_stream


In [10]:
# df.shape

NameError: name 'df' is not defined

In [6]:
question = 'Who is Antonio Neri?'

In [7]:
results = collection.query(query_texts=[question], n_results=5)

date_strings = [i['Date'] for i in results['metadatas'][0]]
# Your list of datetime strings
# date_strings = ['2017-11-21', '2018-03-19', '2022-01-28', '2023-06-20', '2022-04-27']
# Step 1: Parse strings into datetime objects
date_objects = [datetime.fromisoformat(date_str) for date_str in date_strings]

# Step 2: Extract year, month, and day
formatted_dates = [dt.strftime('%Y-%m-%d') for dt in date_objects]

# Step 3: Sort datetime objects while keeping track of original indices
sorted_dates_with_indices = sorted(enumerate(zip(date_objects, formatted_dates)),
                                   key=lambda x: x[1][0], reverse=True)

# Extract sorted dates and original indices
sorted_dates = [date_str for _, (dt, date_str) in sorted_dates_with_indices]
original_indices = [index for index, _ in sorted_dates_with_indices]

# Print the result
print("Sorted Dates:", sorted_dates)
print("Original Indices:", original_indices)
results_x = [results["documents"][0][original_indices[0]],
             results["documents"][0][original_indices[1]],
             results["documents"][0][original_indices[2]]]# get the first three document
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sorted Dates: ['2024-02-14', '2024-01-23', '2024-01-23', '2023-02-22', '2022-10-04']
Original Indices: [0, 2, 4, 1, 3]


{'ids': [['id224', 'id180', 'id222', 'id97', 'id223']],
 'distances': [[0.3126109808262521,
   0.4520028233528137,
   0.4642681158148587,
   0.4674578905105591,
   0.46775119900362405]],
 'metadatas': [[{'Content': 'Antonio Neri is President and Chief Executive Officer of Hewlett Packard Enterprise (HPE), responsible for leading the development and delivery of enterprise technology solutions and services that help organizations accelerate business outcomes by unlocking value from all of their data, everywhere.\n\nSince being named CEO in February 2018, Antonio has leveraged his deep experience as a business leader and technologist to transform HPE into the global edge-to-cloud company. He has led the introduction of leading innovations such as HPE GreenLake, the company’s unified, automated, and secure hybrid-cloud experience, and Frontier, the world’s first supercomputer to break the exascale speed barrier.\n\nAntonio has accelerated the company’s innovation organically and through ta

In [8]:
results_x

['Antonio Neri is President and Chief Executive Officer of Hewlett Packard Enterprise (HPE), responsible for leading the development and delivery of enterprise technology solutions and services that help organizations accelerate business outcomes by unlocking value from all of their data, everywhere.\n\nSince being named CEO in February 2018, Antonio has leveraged his deep experience as a business leader and technologist to transform HPE into the global edge-to-cloud company. He has led the introduction of leading innovations such as HPE GreenLake, the company’s unified, automated, and secure hybrid-cloud experience, and Frontier, the world’s first supercomputer to break the exascale speed barrier.\n\nAntonio has accelerated the company’s innovation organically and through targeted acquisitions, including Aruba Networks, Axis Security, Cray, Determined AI, Nimble Storage, Pachyderm, Silicon Graphics, Silver Peak, and Zerto, among others.\n\nUnder his leadership, HPE has also achieved i

In [9]:
# await show_sources(results)
# print("results: ",results)`
'''
2/6/24 (Andrew): Add limit to ensure that any press release does not exceed >14k. 
This assumes TitanML API deployed on A100
This will decrease when API is deployed no T4.
'''
results2 = "\n".join(results_x)
results2 = results2[:8500]
print("len(results2): ",len(results2))
# print("results2: ",results2)
prompt = f"[INST]`{results2}`. Using the above information, answer the following question: {question}.Answer factually and concisely, answer at most in three sentences. Respond in a natural way, like you are having a conversation with a friend.[/INST]"
print("=========prompt=============: ")
print(prompt)
print("=========end_of_prompt=============")

len(results2):  8500
=========prompt=============: 
[INST]`Antonio Neri is President and Chief Executive Officer of Hewlett Packard Enterprise (HPE), responsible for leading the development and delivery of enterprise technology solutions and services that help organizations accelerate business outcomes by unlocking value from all of their data, everywhere.

Since being named CEO in February 2018, Antonio has leveraged his deep experience as a business leader and technologist to transform HPE into the global edge-to-cloud company. He has led the introduction of leading innovations such as HPE GreenLake, the company’s unified, automated, and secure hybrid-cloud experience, and Frontier, the world’s first supercomputer to break the exascale speed barrier.

Antonio has accelerated the company’s innovation organically and through targeted acquisitions, including Aruba Networks, Axis Security, Cray, Determined AI, Nimble Storage, Pachyderm, Silicon Graphics, Silver Peak, and Zerto, among oth